# Welcome to the Day 2 Lab!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get started --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## First - let's talk about the Chat Completions API

1. The simplest way to call an LLM
2. It's called Chat Completions because it's saying: "here is a conversation, please predict what should come next"
3. The Chat Completions API was invented by OpenAI, but it's so popular that everybody uses it!

### We will start by calling OpenAI again - but don't worry non-OpenAI people, your time is coming!


In [8]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


## Do you know what an Endpoint is?

If not, please review the Technical Foundations guide in the guides folder

And, here is an endpoint that might interest you...

In [2]:
import requests

headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

payload = {
    "model": "gpt-5-nano",
    "messages": [
        {"role": "user", "content": "Tell me a fun fact"}]
}

payload

{'model': 'gpt-5-nano',
 'messages': [{'role': 'user', 'content': 'Tell me a fun fact'}]}

In [4]:
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=payload
)

response.json()

{'id': 'chatcmpl-CRRXbHv1Yb10ytbrdcX01hBZSfX2W',
 'object': 'chat.completion',
 'created': 1760656759,
 'model': 'gpt-5-nano-2025-08-07',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Here are a few fun ones:\n- A group of flamingos is called a flamboyance.\n- Bananas are botanically berries, but strawberries aren’t.\n- Honey has an indefinite shelf life; ancient honey has been found still edible.\n\nWant more, maybe in a specific category (space, animals, history)?',
    'refusal': None,
    'annotations': []},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 11,
  'completion_tokens': 1033,
  'total_tokens': 1044,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 960,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': None}

In [5]:
response.json()["choices"][0]["message"]["content"]

'Here are a few fun ones:\n- A group of flamingos is called a flamboyance.\n- Bananas are botanically berries, but strawberries aren’t.\n- Honey has an indefinite shelf life; ancient honey has been found still edible.\n\nWant more, maybe in a specific category (space, animals, history)?'

# What is the openai package?

It's known as a Python Client Library.

It's nothing more than a wrapper around making this exact call to the http endpoint.

It just allows you to work with nice Python code instead of messing around with janky json objects.

But that's it. It's open-source and lightweight. Some people think it contains OpenAI model code - it doesn't!


In [6]:
# Create OpenAI client

from openai import OpenAI
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content



'Fun fact: A group of flamingos is called a “flamboyance”—and their pink color comes from the carotenoids in their diet.'

## And then this great thing happened:

OpenAI's Chat Completions API was so popular, that the other model providers created endpoints that are identical.

They are known as the "OpenAI Compatible Endpoints".

For example, google made one here: https://generativelanguage.googleapis.com/v1beta/openai/

And OpenAI decided to be kind: they said, hey, you can just use the same client library that we made for GPT. We'll allow you to specify a different endpoint URL and a different key, to use another provider.

So you can use:

```python
gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key="AIz....")
gemini.chat.completions.create(...)
```

And to be clear - even though OpenAI is in the code, we're only using this lightweight python client library to call the endpoint - there's no OpenAI model involved here.

If you're confused, please review Guide 9 in the Guides folder!

And now let's try it!

In [9]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")



API key found and looks good so far!


In [10]:
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

response = gemini.chat.completions.create(model="gemini-2.5-pro", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

'A group of flamingos is called a **flamboyance**.'

## And Ollama also gives an OpenAI compatible endpoint

...and it's on your local machine!

If the next cell doesn't print "Ollama is running" then please open a terminal and run `ollama serve`

In [11]:
requests.get("http://localhost:11434").content

b'Ollama is running'

### Download llama3.2 from meta

Change this to llama3.2:1b if your computer is smaller.

Don't use llama3.3 or llama4! They are too big for your computer..

In [12]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 361 KB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 1.3 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕

In [13]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')


In [14]:
# Get a fun fact

response = ollama.chat.completions.create(model="llama3.2", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

"Did you know that honey never spoils? Archaeologists have discovered jars of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible! Honey's unique composition, which includes water, glucose, and acids, acts as a natural preservative, making it virtually impossible for bacteria or microorganisms to grow and destroy the honey."

In [15]:
# Now let's try deepseek-r1:1.5b - this is DeepSeek "distilled" into Qwen from Alibaba Cloud

!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 886 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 1.1 MB/1.1 GB                  pulling manifest 
pulling a

In [16]:
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

'Sure! Here\'s a fun fact for you: **"All animals have two Dreams."** Each animal possesses two sides of their soul—one happy inside and one inspiring outside. This quote is often attributed to the wise sage Shangnu Xie of China, who practices Tibetan Buddhism. Isn\'t that awesome? 😊'

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [17]:
system_prompt = "You are an email assistant to General Martok, Klingon greatest warrior. You are given an email by your boss and you need to respond to in Klingon as general would."
user_prompt = """
An easy way to save money and help your community
At 3CE, we’re always looking for ways to provide clean energy solutions to our local community and help you keep more cash in your pocket. 

That’s why we partnered with OhmConnect - a free service that will automatically help you lower your electric bill PLUS help keep our community healthy by avoiding polluting power sources.

When electricity prices spike in our area, and we’re in danger of having to fire up a fossil fuel power plant, you’ll get an SMS or email alert to power down some devices. This helps you save energy when it matters most and lowers your bill. You’ll also earn rewards every time you reduce!

To make it even easier, you can link smart devices like a smart thermostat or smart plugs to your OhmConnect account. These devices will then automatically power down when it matters most. 

Sign up now and get $25 just for connecting your utility, it takes just 2 minutes to join!
Enroll now

"""

# Step 2: Make the messages list

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
    ] # fill this in

# Step 3: Call OpenAI
response = ollama.chat.completions.create(model="llama3.2", messages=messages)

# Step 4: print the result
print(response.choices[0].message.content)

**gahwIj DatIvjajwI' Dajatlh'a'**

tIQongwIvaj DatIvjajwI' Dajatlh'a'Dajatlh'.

**Da 'eghla'bogh DatIvjajwI' vIghajbogh tIH Qapla'

VImawDe' ohbogh ghajwIj DatIvjajwI' lIHegh tIQong.

**Dajatlh'a'DaS jajDatIvjajwI' eheH**

tIQong Da 'ejDaQongwIvaj vIS aS jajDatIH. **Qapla'

T'Iv DatIvjajwI' Dajatlh'a', ghajbogh tIH bIng vaDajatlh'.

 (Translation:

The energy is at our hands.

Your connection will be rewarded with 25, and it's easy to do.

Join now, you'll get a prize).

Note: To write in Klingon, I used a combination of words from the Klingon language book "The Klingon Dictionary" by Marc Okrand.


In [19]:
from scraper import fetch_website_contents
from IPython.display import Markdown, display


system_prompt = """
You are a snarkyassistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

def summarize(url):
    website = fetch_website_contents(url)
    response = ollama.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content


def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

display_summary("https://edwarddonner.com")


**Summary:**
This website belongs to Edward Donner, a guy who likes tinkering with Large Language Models (LLMs) on the side. When not DJing or playing with computers, he co-founded Nebula.io, an AI startup that uses LLMs for talent management.

News and Announcements:

* April 2025: "The Complete Agentic AI Engineering Course" was launched.
* May 2025: Courses were made available to help people become LLM experts and leaders.
* May 2025: A course was released on connecting his expertise in LLMs.
* May 2025: An Executive Briefing by Edward Donner.
* September 2024: No major announcement yet